In [1]:
!pip install xlsxwriter pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 8.0 MB/s eta 0:00:00
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=8f5c20aa3ade6cd5c918c13556c2bc64374dbb9c585f2741c261fbc412777042
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive


In [2]:
#%% import
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import json
import re


#%% functions

def get_article_links_from_page(page_url):
    """
    Pobiera wszystkie linki do artykułów z danej strony bloga
    """
    article_links = []
    try:
        r = requests.get(page_url)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'lxml')

        # Opcja 1: Szukamy w elementach <article>
        for article in soup.find_all('article'):
            # Szukamy głównego linku do artykułu
            link = article.find('a', href=True)
            if link and 'jaroslawpluciennik.com' in link['href']:
                href = link['href']
                # Wykluczamy linki do kategorii, tagów, archiwów
                if not any(x in href for x in ['/category/', '/tag/', '/author/', '/page/']):
                    if href not in article_links:
                        article_links.append(href)

        # Opcja 2: Szukamy w divach z klasą post/entry
        if not article_links:
            posts = soup.find_all('div', class_=lambda x: x and ('post' in str(x).lower() or 'entry' in str(x).lower()))
            for post in posts:
                link = post.find('a', href=True)
                if link and 'jaroslawpluciennik.com' in link['href']:
                    href = link['href']
                    if not any(x in href for x in ['/category/', '/tag/', '/author/', '/page/']):
                        if href not in article_links:
                            article_links.append(href)

        # Opcja 3: Szukamy nagłówków z linkami
        if not article_links:
            for heading in soup.find_all(['h1', 'h2', 'h3', 'h4']):
                link = heading.find('a', href=True)
                if link and 'jaroslawpluciennik.com' in link['href']:
                    href = link['href']
                    if not any(x in href for x in ['/category/', '/tag/', '/author/', '/page/']):
                        if href not in article_links:
                            article_links.append(href)

        return article_links

    except Exception as e:
        print(f"Błąd pobierania artykułów z {page_url}: {e}")
        return []


def get_all_pages(blog_url):
    """
    Znajduje wszystkie strony paginacji
    """
    pages = [blog_url]
    try:
        r = requests.get(blog_url)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'lxml')

        # Opcja 1: Szukamy paginacji
        pagination = soup.find('nav', class_=lambda x: x and 'pagination' in str(x).lower())
        if not pagination:
            pagination = soup.find('div', class_=lambda x: x and 'pagination' in str(x).lower())

        if pagination:
            for link in pagination.find_all('a', href=True):
                page_url = link['href']
                # Upewniamy się, że to pełny URL
                if not page_url.startswith('http'):
                    base_url = blog_url.rstrip('/')
                    page_url = base_url + '/' + page_url.lstrip('/')

                if page_url not in pages:
                    pages.append(page_url)

        # Opcja 2: Budujemy strony ręcznie (blog/page/2/, blog/page/3/, etc.)
        if len(pages) == 1:
            # Sprawdzamy czy istnieje strona 2
            test_url = blog_url.rstrip('/') + '/page/2/'
            r = requests.get(test_url)
            if r.status_code == 200:
                page_num = 2
                while True:
                    next_page = blog_url.rstrip('/') + f'/page/{page_num}/'
                    r = requests.get(next_page)
                    if r.status_code == 200:
                        pages.append(next_page)
                        page_num += 1
                        time.sleep(0.5)
                    else:
                        break

        return sorted(set(pages))
    except Exception as e:
        print(f"Błąd przy szukaniu stron: {e}")
        return pages


def get_all_article_links(blog_url):
    """
    Główna funkcja - pobiera wszystkie linki do artykułów
    """
    print("Krok 1: Szukanie wszystkich stron bloga...")
    all_pages = get_all_pages(blog_url)

    print(f"Znaleziono {len(all_pages)} stron")
    for i, page in enumerate(all_pages[:5], 1):
        print(f"  {i}. {page}")
    if len(all_pages) > 5:
        print(f"  ... i {len(all_pages) - 5} więcej")

    all_article_links = []
    errors = []

    print("\nKrok 2: Pobieranie artykułów z każdej strony...")

    for page_url in tqdm(all_pages, desc="Przetwarzanie stron"):
        try:
            time.sleep(0.5)  # Ostrożność
            article_links = get_article_links_from_page(page_url)
            all_article_links.extend(article_links)
        except Exception as e:
            print(f"\nBłąd dla {page_url}: {e}")
            errors.append(page_url)

    # Usuwamy duplikaty
    all_article_links = list(set(all_article_links))

    return all_article_links, errors


#%% main execution

if __name__ == "__main__":
    blog_url = "https://jaroslawpluciennik.com/blog/"

    print("="*60)
    print("Pobieranie wszystkich linków z jaroslawpluciennik.com/blog/")
    print("="*60)

    # Pobierz wszystkie linki
    article_links, errors = get_all_article_links(blog_url)

    # Sortuj alfabetycznie
    article_links.sort()

    # Zapisz do pliku tekstowego
    with open('pluciennik_linki.txt', 'w', encoding='utf-8') as f:
        for link in article_links:
            f.write(link + '\n')

    # Zapisz do JSON (z metadanymi)
    output_data = {
        'source': blog_url,
        'total_links': len(article_links),
        'links': article_links,
        'errors': errors
    }

    with open('pluciennik_linki.json', 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    # Raport
    print("\n" + "="*60)
    print("RAPORT")
    print("="*60)
    print(f"Znaleziono artykułów: {len(article_links)}")
    print(f"Błędów: {len(errors)}")
    if errors:
        print(f"\nProblematyczne URLe:")
        for error_url in errors:
            print(f"  - {error_url}")
    print(f"\nZapisano do:")
    print(f"  - pluciennik_linki.txt")
    print(f"  - pluciennik_linki.json")
    print("="*60)

Pobieranie wszystkich linków z jaroslawpluciennik.com/blog/
Krok 1: Szukanie wszystkich stron bloga...
Znaleziono 40 stron
  1. https://jaroslawpluciennik.com/blog/
  2. https://jaroslawpluciennik.com/blog/page/10/
  3. https://jaroslawpluciennik.com/blog/page/11/
  4. https://jaroslawpluciennik.com/blog/page/12/
  5. https://jaroslawpluciennik.com/blog/page/13/
  ... i 35 więcej

Krok 2: Pobieranie artykułów z każdej strony...


Przetwarzanie stron: 100%|██████████| 40/40 [00:24<00:00,  1.63it/s]


RAPORT
Znaleziono artykułów: 397
Błędów: 0

Zapisano do:
  - pluciennik_linki.txt
  - pluciennik_linki.json


In [3]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    Obsługuje formaty:
    - "09 marzec 2023"
    - "2023-03-09"
    - "09.03.2023"
    """
    try:
        # Usuwamy dodatkowe białe znaki
        date_string = ' '.join(date_string.strip().split())

        # Jeśli już jest w formacie YYYY-MM-DD
        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        # Jeśli jest datetime z czasem
        if 'T' in date_string:
            return date_string.split('T')[0]

        # Słownik z obiema formami miesięcy
        lookup_table = {
            # Dopełniacz
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            # Mianownik
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        # Zamieniamy nazwę miesiąca na numer
        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        # Format DD.MM.YYYY
        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        # Format DD MM YYYY
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        new_date = format(changed_date.date())
        return new_date
    except Exception as e:
        print(f"Błąd konwersji daty '{date_string}': {e}")
        return "no date"


def dictionary_of_article(article_link):
    """
    Pobiera szczegóły artykułu ze strony jaroslawpluciennik.com
    Zwraca dane w tym samym formacie co poprzednie scrapery
    """
    try:
        r = requests.get(article_link)
        r.encoding = 'utf-8'
        html_text = r.text

        # Obsługa rate limiting
        while '429 Too Many Requests' in html_text:
            time.sleep(5)
            r = requests.get(article_link)
            r.encoding = 'utf-8'
            html_text = r.text

        soup = BeautifulSoup(html_text, 'lxml')

        # Data publikacji
        try:
            date_element = soup.find('time')
            if date_element:
                # Próbuj datetime attribute
                date_text = date_element.get('datetime') or date_element.get_text(strip=True)
                date_of_publication = date_change_format(date_text)
            else:
                # Alternatywnie szukaj w spanach/divach z "date"
                date_element = soup.find(['span', 'div'], class_=lambda x: x and 'date' in str(x).lower())
                if date_element:
                    date_text = date_element.get_text(strip=True)
                    date_of_publication = date_change_format(date_text)
                else:
                    date_of_publication = "no date"
        except Exception as e:
            print(f"Błąd parsowania daty dla {article_link}: {e}")
            date_of_publication = "no date"

        # Tytuł
        try:
            title_element = soup.find('h1')
            title = title_element.get_text(strip=True) if title_element else "no title"
        except:
            title = "no title"

        # Autor
        try:
            # Opcja 1: rel="author"
            author_element = soup.find('a', rel='author')
            if not author_element:
                # Opcja 2: klasa z "author"
                author_element = soup.find(['span', 'div', 'a'], class_=lambda x: x and 'author' in str(x).lower())

            if author_element:
                author = author_element.get_text(strip=True)
                # Usuwamy prefix "Autor:", "By:" itp.
                author = re.sub(r'^(Autor|By|Opublikował|Posted by):\s*', '', author, flags=re.IGNORECASE)
            else:
                # Domyślnie Jarosław Pluciennik (to jego blog)
                author = "Jarosław Pluciennik"
        except:
            author = "Jarosław Pluciennik"

        # Treść artykułu
        try:
            # Próbujemy różnych opcji
            article_body = soup.find('div', class_=lambda x: x and 'entry-content' in str(x).lower())
            if not article_body:
                article_body = soup.find('div', class_=lambda x: x and 'post-content' in str(x).lower())
            if not article_body:
                article_body = soup.find('div', class_=lambda x: x and 'article-content' in str(x).lower())
            if not article_body:
                article_body = soup.find('article')

            if article_body:
                text = article_body.get_text(strip=True).replace('\n', ' ').replace('\xa0', ' ')
            else:
                text = "no text"
        except:
            text = "no text"

        # Kategoria
        try:
            category_links = soup.find_all('a', rel='category')
            if not category_links:
                category_links = soup.find_all('a', class_=lambda x: x and 'category' in str(x).lower())

            if category_links:
                categories = [cat.get_text(strip=True) for cat in category_links]
                category = ' | '.join(categories)
            else:
                category = "no category"
        except:
            category = "no category"

        # Linki zewnętrzne
        try:
            if article_body:
                links = [a['href'] for a in article_body.find_all('a', href=True)]
                # Filtrujemy linki wewnętrzne
                external_links = [link for link in links if not re.search(r'jaroslawpluciennik\.com', link)]
                external_links = ' | '.join(external_links) if external_links else None
            else:
                external_links = None
        except (AttributeError, KeyError, IndexError):
            external_links = None

        # Zdjęcia
        try:
            images = []

            # 1. Thumbnail / post-thumbnail (główne zdjęcie artykułu)
            thumbnail_div = soup.find('div', class_=lambda x: x and 'post-thumbnail' in str(x).lower())
            if thumbnail_div:
                thumb_img = thumbnail_div.find('img', src=True)
                if thumb_img:
                    images.append(thumb_img['src'])

            # 2. Featured image (alternatywna nazwa)
            if not images:
                featured_img = soup.find('img', class_=lambda x: x and 'featured' in str(x).lower())
                if not featured_img:
                    featured_div = soup.find('div', class_=lambda x: x and 'featured' in str(x).lower())
                    if featured_div:
                        featured_img = featured_div.find('img')

                if featured_img and featured_img.get('src'):
                    if featured_img['src'] not in images:
                        images.append(featured_img['src'])

            # 3. Zdjęcia w treści artykułu
            if article_body:
                content_images = [img['src'] for img in article_body.find_all('img', src=True)]
                for img_src in content_images:
                    if img_src not in images:
                        images.append(img_src)

            # 4. Inne możliwe miejsca na obrazy (header, figure, etc.)
            for container_class in ['entry-header', 'article-header', 'post-header']:
                header = soup.find('div', class_=container_class)
                if header:
                    header_images = [img['src'] for img in header.find_all('img', src=True)]
                    for img_src in header_images:
                        if img_src not in images:
                            images.append(img_src)

            has_images = len(images) > 0
            photos_links = ' | '.join(images) if images else None
        except (AttributeError, KeyError, IndexError):
            has_images = False
            photos_links = None

        # Filmy (iframe)
        try:
            if article_body:
                iframes = [iframe['src'] for iframe in article_body.find_all('iframe', src=True)]
                has_videos = len(iframes) > 0
            else:
                has_videos = False
        except:
            has_videos = False

        dictionary_of_article = {
            "Link": article_link,
            "Data publikacji": date_of_publication,
            "Tytuł artykułu": title.replace('\xa0', ' '),
            "Tekst artykułu": text,
            "Autor": author,
            "Kategoria": category,
            "Linki zewnętrzne": external_links,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(dictionary_of_article)

    except AttributeError as e:
        errors.append(article_link)
        print(f"Błąd dla {article_link}: {e}")
    except Exception as e:
        errors.append(article_link)
        print(f"Nieoczekiwany błąd dla {article_link}: {e}")


#%% main execution

if __name__ == "__main__":
    # Wczytaj linki z pliku
    try:
        with open('pluciennik_linki.txt', 'r', encoding='utf-8') as f:
            article_links = [line.strip() for line in f if line.strip()]
        print(f"Wczytano {len(article_links)} linków z pliku")
    except FileNotFoundError:
        print("Nie znaleziono pliku pluciennik_linki.txt")
        print("Użyj najpierw get_pluciennik_links.py aby pobrać linki!")
        print("\nLub podaj linki ręcznie:")
        article_links = [
            # Wstaw tutaj linki do artykułów
            # "https://jaroslawpluciennik.com/blog/artykul1/",
            # "https://jaroslawpluciennik.com/blog/artykul2/",
        ]

    if not article_links:
        print("Brak linków do przetworzenia!")
        exit(1)

    all_results = []
    errors = []

    print("\n" + "="*60)
    print("Rozpoczynam scraping artykułów z jaroslawpluciennik.com")
    print("="*60 + "\n")

    # Scraping z progress barem
    with ThreadPoolExecutor(max_workers=5) as executor:
        list(tqdm(executor.map(dictionary_of_article, article_links), total=len(article_links)))

    # Zapisywanie wyników
    timestamp = datetime.today().date()

    # JSON
    with open(f'pluciennik_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)

    # Excel
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(f"pluciennik_{timestamp}.xlsx",
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)

    # Raport
    print(f"\n{'='*60}")
    print(f"Scraping zakończony!")
    print(f"Przetworzono artykułów: {len(all_results)}")
    print(f"Błędów: {len(errors)}")
    if errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        if len(errors) > 10:
            print(f"  ... i {len(errors) - 10} więcej")
    print(f"\nPliki wyjściowe:")
    print(f"  - pluciennik_{timestamp}.json")
    print(f"  - pluciennik_{timestamp}.xlsx")
    print(f"{'='*60}\n")

Wczytano 397 linków z pliku

Rozpoczynam scraping artykułów z jaroslawpluciennik.com



100%|██████████| 397/397 [01:47<00:00,  3.69it/s]
/tmp/ipython-input-455744910.py:299: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Posts', index=False)



Scraping zakończony!
Przetworzono artykułów: 397
Błędów: 0

Pliki wyjściowe:
  - pluciennik_2026-01-12.json
  - pluciennik_2026-01-12.xlsx



In [4]:
df.head()

,Link,Data publikacji,Tytuł artykułu,Tekst artykułu,Autor,Kategoria,Linki zewnętrzne,Zdjęcia/Grafika,Filmy,Linki do zdjęć
0,https://jaroslawpluciennik.com/2019/07/22/gosc...,2019-07-22,Gościu siądź pod mym liściem… tęczowym,Lipa nad Wełtawą w Pradze — symbol CzechGościu...,Jarosław Pluciennik,różnorodność | tolerancja | Uniwersytet,https://www.eua.eu/101-projects/737-invited.html,True,False,https://jaroslawpluciennik.com/wp-content/uplo...
1,https://jaroslawpluciennik.com/2019/08/12/else...,2019-08-12,"Elsevier, kalifornijska odwaga i polskie przyg...",Jak podaje „California Academics Quit Elsevier...,Jarosław Pluciennik,nauka | Uniwersytet,https://www.timeshighereducation.com/news/cali...,True,False,https://jaroslawpluciennik.com/wp-content/uplo...
2,https://jaroslawpluciennik.com/2019/08/14/gwia...,2019-08-14,Gwiazda literaturoznawcza za 200 punktów!!!,Przeanalizowałem listę czasopism w wykazie min...,w. Bialikpisze:,Uniwersytet,http://www.antigonishreview.com/,True,False,https://jaroslawpluciennik.com/wp-content/uplo...
3,https://jaroslawpluciennik.com/2019/07/25/swie...,2019-07-25,Święto Łodzi i orgia,Witraż z siedziby Akademii Muzycznej w Łodzi p...,Jarosław Pluciennik,doktor honoris causa | pojednanie między narod...,http://creativecommons.org/licenses/by-sa/3.0/,True,False,https://jaroslawpluciennik.com/wp-content/uplo...
4,https://jaroslawpluciennik.com/2019/08/19/bum-...,2019-08-19,BUM! Baza otwartych sylabusów świata. Może now...,Open education concept. Getting education onli...,Jarosław Pluciennik,Uniwersytet,http://„Can Mapping Curricula Shed Light on Te...,True,False,https://jaroslawpluciennik.com/wp-content/uplo...
